# AutoGen Simplified Multi-Agent Research System

This is a **simplified and clean** version that removes all fallback mechanisms and focuses on a properly configured SelectorGroupChat for reliable multi-agent coordination.

## Key Improvements

1. **Simplified Configuration**: Single model (Claude 3.5 Sonnet) for all agents to eliminate compatibility issues
2. **Clear Agent Handoffs**: Explicit handoff signals between agents 
3. **Optimized Selector Prompt**: Better agent selection logic
4. **No Fallback Complexity**: Clean, focused implementation

## Architecture
- **Planning Agent**: Creates research plans and identifies sources
- **Web Search Agent**: Scrapes websites for content  
- **Citation Agent**: Validates sources and creates citations
- **Finalize Agent**: Creates comprehensive final reports

In [1]:
# Install required packages
%pip install -U "autogen-agentchat" "autogen-ext[openai]" requests beautifulsoup4 python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import asyncio
import requests
from typing import List, Dict, Any
from datetime import datetime
from bs4 import BeautifulSoup
from dotenv import load_dotenv

# AutoGen v0.4 imports
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelInfo, ModelFamily

# Load environment variables
load_dotenv()

print("Environment setup complete!")

Environment setup complete!


In [7]:
# Simplified Configuration - Using Claude 3.5 Sonnet for all agents for consistency
SCRAPINGDOG_API_KEY = os.getenv("SCRAPINGDOG_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_BASE_URL = os.getenv("OPENROUTER_BASE_URL", "https://openrouter.ai/api/v1")

# Use single reliable model for all agents
MODEL_NAME = "anthropic/claude-sonnet-4"

# Validate required environment variables
required_vars = {
    "SCRAPINGDOG_API_KEY": SCRAPINGDOG_API_KEY,
    "OPENROUTER_API_KEY": OPENROUTER_API_KEY
}

missing_vars = [var for var, value in required_vars.items() if not value]
if missing_vars:
    raise ValueError(f"Missing required environment variables: {', '.join(missing_vars)}")

print("Configuration validated successfully!")
print(f"Using model: {MODEL_NAME} for all agents")
print(f"OpenRouter Base URL: {OPENROUTER_BASE_URL}")

Configuration validated successfully!
Using model: anthropic/claude-sonnet-4 for all agents
OpenRouter Base URL: https://openrouter.ai/api/v1


In [ ]:
# ScrapingDog API Client with Google Search functionality
class ScrapingDogClient:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.scrape_url_endpoint = "https://api.scrapingdog.com/scrape"
        self.google_search_endpoint = "https://api.scrapingdog.com/google_search"
        self.google_scholar_endpoint = "https://api.scrapingdog.com/google_scholar"
        self.google_news_endpoint = "https://api.scrapingdog.com/google_news"
    
    def search_google(self, query: str, num_results: int = 10, country: str = "us") -> Dict[str, Any]:
        """Search Google using ScrapingDog Google Search API"""
        params = {
            "api_key": self.api_key,
            "q": query,
            "num": num_results,
            "gl": country
        }
        
        try:
            response = requests.get(self.google_search_endpoint, params=params, timeout=60)
            
            if response.status_code == 200:
                search_data = response.json()
                urls = []
                
                # Extract URLs from organic results
                if 'organic_results' in search_data:
                    for result in search_data['organic_results'][:num_results]:
                        urls.append({
                            "url": result.get('link', ''),
                            "title": result.get('title', ''),
                            "snippet": result.get('snippet', ''),
                            "rank": result.get('rank', 0)
                        })
                
                return {
                    "success": True,
                    "query": query,
                    "urls": urls,
                    "total_results": len(urls),
                    "searched_at": datetime.now().isoformat()
                }
            else:
                return {
                    "success": False,
                    "query": query,
                    "error": f"Search failed: HTTP {response.status_code}",
                    "searched_at": datetime.now().isoformat()
                }
                
        except Exception as e:
            return {
                "success": False,
                "query": query,
                "error": f"Search error: {str(e)}",
                "searched_at": datetime.now().isoformat()
            }
    
    def search_google_scholar(self, query: str, num_results: int = 10) -> Dict[str, Any]:
        """Search Google Scholar using ScrapingDog API"""
        params = {
            "api_key": self.api_key,
            "q": query,
            "num": num_results
        }
        
        try:
            response = requests.get(self.google_scholar_endpoint, params=params, timeout=60)
            
            if response.status_code == 200:
                scholar_data = response.json()
                urls = []
                
                # Extract URLs from scholar results
                if 'organic_results' in scholar_data:
                    for result in scholar_data['organic_results'][:num_results]:
                        if result.get('link'):
                            urls.append({
                                "url": result.get('link', ''),
                                "title": result.get('title', ''),
                                "snippet": result.get('snippet', ''),
                                "authors": result.get('authors', ''),
                                "year": result.get('year', ''),
                                "citations": result.get('citations', ''),
                                "type": "academic"
                            })
                
                return {
                    "success": True,
                    "query": query,
                    "urls": urls,
                    "total_results": len(urls),
                    "searched_at": datetime.now().isoformat()
                }
            else:
                return {
                    "success": False,
                    "query": query,
                    "error": f"Scholar search failed: HTTP {response.status_code}",
                    "searched_at": datetime.now().isoformat()
                }
                
        except Exception as e:
            return {
                "success": False,
                "query": query,
                "error": f"Scholar search error: {str(e)}",
                "searched_at": datetime.now().isoformat()
            }
    
    def search_google_news(self, query: str, num_results: int = 10) -> Dict[str, Any]:
        """Search Google News using ScrapingDog API"""
        params = {
            "api_key": self.api_key,
            "q": query,
            "num": num_results
        }
        
        try:
            response = requests.get(self.google_news_endpoint, params=params, timeout=60)
            
            if response.status_code == 200:
                news_data = response.json()
                urls = []
                
                # Extract URLs from news results
                if 'news_results' in news_data:
                    for result in news_data['news_results'][:num_results]:
                        urls.append({
                            "url": result.get('link', ''),
                            "title": result.get('title', ''),
                            "snippet": result.get('snippet', ''),
                            "source": result.get('source', ''),
                            "date": result.get('date', ''),
                            "type": "news"
                        })
                
                return {
                    "success": True,
                    "query": query,
                    "urls": urls,
                    "total_results": len(urls),
                    "searched_at": datetime.now().isoformat()
                }
            else:
                return {
                    "success": False,
                    "query": query,
                    "error": f"News search failed: HTTP {response.status_code}",
                    "searched_at": datetime.now().isoformat()
                }
                
        except Exception as e:
            return {
                "success": False,
                "query": query,
                "error": f"News search error: {str(e)}",
                "searched_at": datetime.now().isoformat()
            }
    
    def scrape_url(self, url: str, render_js: bool = True, country: str = "US") -> Dict[str, Any]:
        """Scrape a URL using ScrapingDog API"""
        params = {
            "api_key": self.api_key,
            "url": url,
            "dynamic": "true" if render_js else "false",
            "country": country
        }
        
        try:
            response = requests.get(self.scrape_url_endpoint, params=params, timeout=60)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                text_content = soup.get_text(separator=' ', strip=True)
                title = soup.find('title')
                title_text = title.get_text().strip() if title else "No title found"
                meta_desc = soup.find('meta', attrs={'name': 'description'})
                description = meta_desc.get('content', '') if meta_desc else ''
                
                return {
                    "success": True,
                    "url": url,
                    "title": title_text,
                    "description": description,
                    "content": text_content[:5000],
                    "scraped_at": datetime.now().isoformat()
                }
            else:
                return {
                    "success": False,
                    "url": url,
                    "error": f"HTTP {response.status_code}: {response.text}",
                    "scraped_at": datetime.now().isoformat()
                }
                
        except Exception as e:
            return {
                "success": False,
                "url": url,
                "error": str(e),
                "scraped_at": datetime.now().isoformat()
            }

# Initialize ScrapingDog client
scraping_client = ScrapingDogClient(SCRAPINGDOG_API_KEY)
print("ScrapingDog client initialized!")

ScrapingDog client initialized!


In [8]:
# Create OpenRouter client
def create_openrouter_client(model_name: str) -> OpenAIChatCompletionClient:
    """Create an OpenRouter client for a specific model"""
    model_info = ModelInfo(
        vision=False,
        function_calling=True,
        json_output=True,
        family=ModelFamily.CLAUDE_4_SONNET,
        structured_output=True
    )
    
    return OpenAIChatCompletionClient(
        model=model_name,
        api_key=OPENROUTER_API_KEY,
        base_url=OPENROUTER_BASE_URL,
        model_info=model_info
    )

# Create model client (same for all agents)
print("Creating OpenRouter model client...")
model_client = create_openrouter_client(MODEL_NAME)
print(f"Model client created successfully: {MODEL_NAME}")

Creating OpenRouter model client...
Model client created successfully: anthropic/claude-sonnet-4


In [2]:
# Web scraping and search tools
def search_web_sources(query: str, include_academic: bool = True, include_news: bool = True) -> str:
    """Search for relevant web sources using ScrapingDog Google Search APIs."""
    results = []
    all_urls = []
    
    try:
        # Search general web sources
        web_search = scraping_client.search_google(query, num_results=5)
        if web_search["success"]:
            results.append(f"WEB SEARCH RESULTS ({len(web_search['urls'])} found):")
            for i, url_info in enumerate(web_search['urls'], 1):
                results.append(f"{i}. {url_info['title']}")
                results.append(f"   URL: {url_info['url']}")
                results.append(f"   Snippet: {url_info['snippet'][:200]}...")
                all_urls.append(url_info['url'])
            results.append("")
        else:
            results.append(f"WEB SEARCH FAILED: {web_search['error']}")
        
        # Search academic sources if requested
        if include_academic:
            scholar_search = scraping_client.search_google_scholar(query, num_results=3)
            if scholar_search["success"]:
                results.append(f"ACADEMIC SEARCH RESULTS ({len(scholar_search['urls'])} found):")
                for i, url_info in enumerate(scholar_search['urls'], 1):
                    results.append(f"{i}. {url_info['title']}")
                    if url_info.get('authors'):
                        results.append(f"   Authors: {url_info['authors']}")
                    if url_info.get('year'):
                        results.append(f"   Year: {url_info['year']}")
                    results.append(f"   URL: {url_info['url']}")
                    all_urls.append(url_info['url'])
                results.append("")
            else:
                results.append(f"ACADEMIC SEARCH FAILED: {scholar_search['error']}")
        
        # Search news sources if requested
        if include_news:
            news_search = scraping_client.search_google_news(query, num_results=3)
            if news_search["success"]:
                results.append(f"NEWS SEARCH RESULTS ({len(news_search['urls'])} found):")
                for i, url_info in enumerate(news_search['urls'], 1):
                    results.append(f"{i}. {url_info['title']}")
                    if url_info.get('source'):
                        results.append(f"   Source: {url_info['source']}")
                    if url_info.get('date'):
                        results.append(f"   Date: {url_info['date']}")
                    results.append(f"   URL: {url_info['url']}")
                    all_urls.append(url_info['url'])
                results.append("")
            else:
                results.append(f"NEWS SEARCH FAILED: {news_search['error']}")
        
        # Summary
        results.append(f"SEARCH SUMMARY:")
        results.append(f"- Total URLs found: {len(all_urls)}")
        results.append(f"- Search completed at: {datetime.now().isoformat()}")
        results.append(f"- Next step: Use scrape_website() to extract content from relevant URLs")
        
        return "\n".join(results)
        
    except Exception as e:
        return f"""SEARCH ERROR for query: {query}
ERROR: {str(e)}
TIMESTAMP: {datetime.now().isoformat()}
STATUS: Search failed, please try again with a different query."""

def scrape_website(url: str, render_js: bool = True, country: str = "US") -> str:
    """Scrape a website using the ScrapingDog API."""
    result = scraping_client.scrape_url(url, render_js, country)
    
    if result["success"]:
        return f"""SCRAPED CONTENT FROM: {url}

TITLE: {result['title']}
DESCRIPTION: {result['description']}
CONTENT: {result['content']}
SCRAPED AT: {result['scraped_at']}

STATUS: Content successfully extracted."""
    else:
        return f"""SCRAPING FAILED FOR: {url}
ERROR: {result['error']}
SCRAPED AT: {result['scraped_at']}
STATUS: Please try a different URL."""

def create_research_plan(query: str) -> str:
    """Create a structured research plan for a given query."""
    return f"""RESEARCH PLAN FOR: {query}
Generated: {datetime.now().isoformat()}

PHASE 1: Source Discovery & Identification
- Use search_web_sources() to find relevant URLs across:
  * General web sources (Google Search)
  * Academic sources (Google Scholar)
  * News sources (Google News)
- Focus on recent, authoritative sources

PHASE 2: Content Extraction & Analysis
- Use scrape_website() to extract content from discovered URLs
- Analyze content for key facts, insights, and data points
- Extract relevant quotes and statistics

PHASE 3: Source Validation & Citation Management
- Verify source credibility and authority
- Check publication dates and author credentials
- Create properly formatted APA citations
- Identify any bias or limitations

PHASE 4: Synthesis & Final Report
- Synthesize findings from all sources
- Create comprehensive analysis with evidence-based conclusions
- Structure report with executive summary, methodology, findings, and recommendations

METHODOLOGY: Web-first research using ScrapingDog APIs for real-time content discovery and extraction
READY FOR IMPLEMENTATION"""

print("Web search and scraping tools defined successfully!")
print("- search_web_sources(): Performs real Google searches using ScrapingDog APIs")
print("- scrape_website(): Extracts content from URLs")
print("- create_research_plan(): Creates structured research plans")

Web search and scraping tools defined successfully!
- search_web_sources(): Performs real Google searches using ScrapingDog APIs
- scrape_website(): Extracts content from URLs
- create_research_plan(): Creates structured research plans


In [10]:
# Create agents with real web search functionality
planning_agent = AssistantAgent(
    name="PlanningAgent",
    model_client=model_client,
    tools=[create_research_plan, search_web_sources],
    description="Creates structured research plans and discovers relevant web sources using real-time search.",
    system_message="""You are a Planning Agent specialized in research planning and web source discovery.

Your responsibilities:
1. Create comprehensive research plans using create_research_plan()
2. Discover relevant sources using search_web_sources() - this performs REAL Google searches
3. Establish research methodology and objectives
4. Provide URLs for content extraction

Process:
- Analyze the research query to understand objectives
- Create structured research plan with phases
- Use search_web_sources() to find actual, relevant URLs from:
  * Google Search (general web sources)
  * Google Scholar (academic sources)  
  * Google News (current news sources)
- End with "Research plan complete. Now we need to gather content from these sources."

Always start with "RESEARCH PLAN:" and use search_web_sources() to find real URLs, then conclude with clear handoff to content gathering phase."""
)

web_search_agent = AssistantAgent(
    name="WebSearchAgent",
    model_client=model_client,
    tools=[scrape_website],
    description="Scrapes websites and extracts content from URLs using ScrapingDog API.",
    system_message="""You are a Web Search Agent specialized in content extraction and information gathering.

Your responsibilities:
1. Use scrape_website() to extract content from identified URLs
2. Analyze and synthesize key findings from multiple sources
3. Extract relevant data, facts, and insights for the research topic
4. Organize information for citation and validation

Process:
- Extract content from each identified source systematically
- Summarize key findings and relevant information
- Note important quotes, statistics, and claims
- End with "Content extracted, ready for citation review and source validation."

Always start with "SEARCH RESULTS:" and conclude with clear handoff to citation validation phase."""
)

citation_agent = AssistantAgent(
    name="CitationAgent",
    model_client=model_client,
    description="Validates sources and creates proper APA citations for research.",
    system_message="""You are a Citation Agent specialized in source validation and academic integrity.

Your responsibilities:
1. Review and validate all sources used in research
2. Assess source credibility, authority, and reliability
3. Create properly formatted APA citations
4. Identify potential bias or limitations in sources

Process:
- Evaluate each source for credibility and relevance
- Check publication dates, author credentials, and publisher reputation
- Format citations according to APA standards
- Note any source limitations or potential bias
- End with "Sources verified and citations formatted. Ready for final synthesis."

Always start with "CITATION REVIEW:" and conclude with clear handoff to final report synthesis."""
)

finalize_agent = AssistantAgent(
    name="FinalizeAgent",
    model_client=model_client,
    description="Synthesizes research findings into comprehensive final reports.",
    system_message="""You are a Finalization Agent specialized in research synthesis and report creation.

Your responsibilities:
1. Synthesize all research findings from previous agents
2. Create comprehensive, well-structured final reports
3. Provide analytical insights and evidence-based conclusions
4. Include executive summary, methodology, findings, and recommendations

Process:
- Review all research findings and validated sources
- Synthesize information into coherent narrative
- Provide analytical insights and evidence-based conclusions
- Structure report with executive summary, methodology, key findings, analysis, and recommendations
- Ensure all sources are properly cited

Always start with "FINAL REPORT:" and create a complete, professional research document."""
)

print("All agents created with real web search functionality!")
print("- PlanningAgent: Uses search_web_sources() for real-time web discovery")
print("- WebSearchAgent: Uses scrape_website() for content extraction")
print("- CitationAgent: Validates sources and creates citations")
print("- FinalizeAgent: Creates comprehensive final reports")


All agents created with real web search functionality!
- PlanningAgent: Uses search_web_sources() for real-time web discovery
- WebSearchAgent: Uses scrape_website() for content extraction
- CitationAgent: Validates sources and creates citations
- FinalizeAgent: Creates comprehensive final reports


In [11]:
# Create SelectorGroupChat with OPTIMIZED selector prompt using history, roles, participants
agent_team = SelectorGroupChat(
    participants=[
        planning_agent,
        web_search_agent,
        citation_agent,
        finalize_agent
    ],
    model_client=model_client,
    termination_condition=MaxMessageTermination(max_messages=15),
    allow_repeated_speaker=True,  # Essential for flexibility
    selector_prompt="""Select the next agent based on the current conversation context and workflow progression.

Available agents and their capabilities:
{roles}

Conversation history:
{history}

Based on the conversation above, select the most appropriate agent from {participants} to handle the next task.

Selection Guidelines:
- If research planning is needed or sources need to be identified → PlanningAgent
- If websites need to be scraped or content needs to be extracted → WebSearchAgent  
- If sources need validation or citations need to be created → CitationAgent
- If all information is gathered and a final report needs to be synthesized → FinalizeAgent

Analyze the conversation flow and choose the agent whose expertise best matches what needs to happen next.

Selected agent:"""
)

print("SelectorGroupChat created with OPTIMIZED selector prompt!")
print("Key improvements:")
print("✅ Uses {history} to analyze conversation context")
print("✅ Uses {roles} to understand agent capabilities") 
print("✅ Uses {participants} for available agent selection")
print("✅ Provides clear workflow guidance")
print("✅ Maintains flexibility with allow_repeated_speaker=True")

SelectorGroupChat created with OPTIMIZED selector prompt!
Key improvements:
✅ Uses {history} to analyze conversation context
✅ Uses {roles} to understand agent capabilities
✅ Uses {participants} for available agent selection
✅ Provides clear workflow guidance
✅ Maintains flexibility with allow_repeated_speaker=True


In [12]:
# Research function with fallback strategy (best practice)
async def run_research(research_query: str):
    """Run research with SelectorGroupChat and RoundRobin fallback"""
    print(f"\nStarting Research: {research_query}")
    print("=" * 80)
    
    task_message = f"""Please conduct comprehensive research on: {research_query}

This is a collaborative multi-agent research session:
- Planning Agent: Create research plan and identify sources
- Web Search Agent: Extract content from websites
- Citation Agent: Validate sources and create citations
- Finalize Agent: Create comprehensive final report

Work together to produce high-quality research with proper citations."""
    
    message_count = 0
    agents_participated = set()
    has_final_report = False
    
    try:
        print("🧠 Using SelectorGroupChat (intelligent agent selection)...")
        async for message in agent_team.run_stream(task=task_message):
            message_count += 1
            
            if hasattr(message, 'source') and hasattr(message, 'content'):
                agent_name = message.source
                agents_participated.add(agent_name)
                
                if "FINAL REPORT:" in message.content:
                    has_final_report = True
                
                print(f"\n[{agent_name}] (Message {message_count}):")
                print("-" * 50)
                print(message.content)
                print()
                
            elif str(type(message).__name__) == 'TaskResult':
                print(f"\n[TASK RESULT]: Session completed after {message_count} messages")
                break
                
    except Exception as e:
        print(f"SelectorGroupChat error: {e}")
        print("🔄 Falling back to RoundRobin approach...")
        
        # Create fallback RoundRobin team
        from autogen_agentchat.teams import RoundRobinGroupChat
        fallback_team = RoundRobinGroupChat(
            participants=[planning_agent, web_search_agent, citation_agent, finalize_agent],
            termination_condition=MaxMessageTermination(max_messages=12)
        )
        
        try:
            message_count = 0
            agents_participated = set()
            has_final_report = False
            
            async for message in fallback_team.run_stream(task=task_message):
                message_count += 1
                
                if hasattr(message, 'source') and hasattr(message, 'content'):
                    agent_name = message.source
                    agents_participated.add(agent_name)
                    
                    if "FINAL REPORT:" in message.content:
                        has_final_report = True
                    
                    print(f"\n[{agent_name}] (Message {message_count}):")
                    print("-" * 50)
                    print(message.content)
                    print()
                    
                elif str(type(message).__name__) == 'TaskResult':
                    print(f"\n[FALLBACK RESULT]: RoundRobin completed after {message_count} messages")
                    break
                    
        except Exception as e2:
            print(f"Fallback error: {e2}")
    
    # Summary
    print("\n" + "=" * 80)
    print("Research Session Complete!")
    print(f"Total messages: {message_count}")
    print(f"Agents participated: {', '.join(sorted(agents_participated))}")
    print(f"Final report generated: {'Yes' if has_final_report else 'No'}")
    
    success = len(agents_participated) >= 3 and has_final_report
    if success:
        print("✅ SUCCESS: Research completed successfully!")
    else:
        print("⚠️ PARTIAL: Research had some issues")
        print("Recommendations:")
        if len(agents_participated) < 3:
            print("- Increase max_messages to allow more agent participation")
        if not has_final_report:
            print("- Ensure FinalizeAgent creates final report")
    
    return success

print("Research function ready with SelectorGroupChat + RoundRobin fallback!")

Research function ready with SelectorGroupChat + RoundRobin fallback!


In [ ]:
# Test with PROVEN best practices approach
research_topic = "Latest genai RAG techniques and their impact on IT development industry in 2025"

print("🚀 Testing AutoGen Research System with PROVEN Best Practices")
# print("=" * 70)
# print("Based on research of working AutoGen v0.4 implementations:")
# print("✅ Simple, context-based selector prompt")
# print("✅ Clear agent descriptions")
# print("✅ Flexible agent selection (allow_repeated_speaker=True)")
# print("✅ RoundRobin fallback strategy")
# print("✅ Reasonable termination conditions")
# print()
# print("Expected behavior: Intelligent agent selection based on context")
# print("Fallback: If SelectorGroupChat fails, automatically use RoundRobin")
# print()

success = await run_research(research_topic)

print(f"\n🎯 Final Result: {'✅ SUCCESS' if success else '❌ NEEDS IMPROVEMENT'}")

if not success:
    print("\n💡 Next steps if this still doesn't work:")
    print("1. Try using GPT-4o instead of Claude 3.5 Sonnet for selection")
    print("2. Reduce max_messages to 8-10 to force quicker decisions")
    print("3. Consider using RoundRobin as primary approach")
    print("4. Check if API keys/models are working correctly")

🚀 Testing AutoGen Research System with PROVEN Best Practices

Starting Research: Latest genai RAG techniques and their impact on IT development industry in 2025
🧠 Using SelectorGroupChat (intelligent agent selection)...

[user] (Message 1):
--------------------------------------------------
Please conduct comprehensive research on: Latest genai RAG techniques and their impact on IT development industry in 2025

This is a collaborative multi-agent research session:
- Planning Agent: Create research plan and identify sources
- Web Search Agent: Extract content from websites
- Citation Agent: Validate sources and create citations
- Finalize Agent: Create comprehensive final report

Work together to produce high-quality research with proper citations.


[PlanningAgent] (Message 2):
--------------------------------------------------
RESEARCH PLAN:

I'll create a comprehensive research plan for the latest GenAI RAG techniques and their impact on IT development in 2025, then identify relevant

In [ ]:
# Test with CONTEXT-AWARE selector prompt using history/roles/participants
research_topic = "Latest developments in quantum computing and their impact on cybersecurity"

print("🚀 Testing AutoGen with CONTEXT-AWARE Selector Prompt")
print("=" * 70)
print("New approach uses AutoGen v0.4 template variables:")
print("✅ {history} - Full conversation context for intelligent decisions")
print("✅ {roles} - Agent descriptions and capabilities")
print("✅ {participants} - Available agents for selection")
print("✅ Context-aware workflow progression")
print()
print("Expected behavior: Selector analyzes conversation and chooses appropriate agent")
print("No more rigid handoff signals - intelligent context-based selection")
print()

success = await run_research(research_topic)

print(f"\n🎯 Final Result: {'✅ SUCCESS' if success else '❌ NEEDS IMPROVEMENT'}")

if success:
    print("🎉 The context-aware selector prompt is working!")
    print("The selector successfully used conversation history to choose appropriate agents.")
else:
    print("\n💡 If this still doesn't work, try these alternatives:")
    print("1. Use GPT-4 or Claude 4 Sonnet for the selector model (stronger reasoning)")
    print("2. Reduce max_messages to 8-10 for faster decisions")
    print("3. The RoundRobin fallback should still work reliably")
    print("4. Check that all agents have clear, descriptive descriptions")

In [ ]:
# Test the ENFORCED handoff system
research_topic = "Latest developments in quantum computing and their impact on cybersecurity"

print("Testing ENFORCED HANDOFF AutoGen Research System")
print("This version uses aggressive handoff enforcement and prevents agent repetition")
print("\nExpected workflow: Planning -> WebSearch -> Citation -> Finalize")
print("Each agent MUST hand off to the next agent exactly once")

# Create explicit task message that reinforces handoffs
task_message = f"""URGENT RESEARCH REQUEST: {research_topic}

MANDATORY WORKFLOW SEQUENCE (MUST FOLLOW EXACTLY):
1. PlanningAgent: Create plan, identify sources, then say "HANDOFF TO WEBSEARCH AGENT"
2. WebSearchAgent: Scrape content, extract findings, then say "HANDOFF TO CITATION AGENT" 
3. CitationAgent: Review sources, create citations, then say "HANDOFF TO FINALIZE AGENT"
4. FinalizeAgent: Create final report starting with "FINAL REPORT:"

CRITICAL: Each agent speaks exactly once and MUST use the handoff signal to pass control.
This is a STRICT sequential workflow - no deviations allowed."""

print(f"\nStarting Research: {research_topic}")
print("=" * 80)

message_count = 0
agents_participated = set()
has_final_report = False

try:
    async for message in agent_team.run_stream(task=task_message):
        message_count += 1
        
        if hasattr(message, 'source') and hasattr(message, 'content'):
            agent_name = message.source
            agents_participated.add(agent_name)
            
            if "FINAL REPORT:" in message.content:
                has_final_report = True
            
            print(f"\n[{agent_name}] (Message {message_count}):")
            print("-" * 50)
            print(message.content)
            print()
            
            # Show handoff detection
            if "HANDOFF TO" in message.content:
                handoff_line = [line for line in message.content.split('\n') if 'HANDOFF TO' in line]
                if handoff_line:
                    print(f"🔄 HANDOFF DETECTED: {handoff_line[0].strip()}")
            
        elif str(type(message).__name__) == 'TaskResult':
            print(f"\n[TASK RESULT]: Session completed after {message_count} messages")
            break
            
except Exception as e:
    print(f"Research error: {e}")

# Summary
print("\n" + "=" * 80)
print("Research Session Complete!")
print(f"Total messages: {message_count}")
print(f"Agents participated: {', '.join(sorted(agents_participated))}")
print(f"Final report generated: {'Yes' if has_final_report else 'No'}")

if len(agents_participated) >= 3 and has_final_report:
    print("SUCCESS: Research completed successfully!")
else:
    print("PARTIAL: Research had some issues")

success = has_final_report and len(agents_participated) >= 3
print(f"\nResearch Success: {success}")